# GAN



## 1、GAN(Generative Adversarial Networks) 是什么？

GAN包含两个网络，一个是generator，一个是discriminator，两个网络相互之间进行对抗，来达到最好的生成效果


## 2、GAN的原理是什么？

GAN的基本原理其实非常简单，这里以生成图片为例进行说明。

假设我们有两个网络，G（Generator）和D（Discriminator）。正如它的名字所暗示的那样，它们的功能分别是：

* G是一个生成图片的网络，它接收一个随机的噪声z，通过这个噪声生成图片，记做G(z)。

* D是一个判别网络，判别一张图片是不是“真实的”。它的输入参数是x，x代表一张图片，输出D（x）代表x为真实图片的概率，如果为1，就代表100%是真实的图片，而输出为0，就代表不可能是真实的图片。

在训练过程中，**生成网络G的目标就是尽量生成真实的图片去欺骗判别网络D。而D的目标就是尽量把G生成的图片和真实的图片分别开来。**

这样，G和D构成了一个动态的“博弈过程”。


下面这个图就描述了这个过程


![GAN](./images/GAN/GAN.jpg)




## 3、 如何训练GAN呢？

我们已经知道了GAN可以看成一个“博弈过程”,它的目标是优化函数：


![GAN equal](./images/GAN/GAN_eq.jpg)



简单分析一下这个公式：

* 整个式子由两项构成。x表示真实图片，z表示输入G网络的噪声，而G(z)表示G网络生成的图片。

* D(x)表示D网络判断真实图片是否真实的概率（因为x就是真实的，所以对于D来说，这个值越接近1越好）。而D(G(z))是D网络判断G生成的图片的是否真实的概率。

* G的目的：上面提到过，D(G(z))是D网络判断G生成的图片是否真实的概率，G应该希望自己生成的图片“越接近真实越好”。也就是说，G希望D(G(z))尽可能得大，这时V(D, G)会变小。因此我们看到式子的最前面的记号是min_G。

* D的目的：D的能力越强，D(x)应该越大，D(G(x))应该越小。这时V(D,G)会变大。因此式子对于D来说是求最大(max_D)


上面的公式是论文中的公式，在实际应用中，我们会使用如下公式：


**Discriminator 的update**

![GAN D](./images/GAN/D.png)



**Generator 的update**

![GAN G](./images/GAN/G.png)





以上内容参考 https://zhuanlan.zhihu.com/p/24767059 和 李宏毅老师PPT http://speech.ee.ntu.edu.tw/~tlkagk/courses_MLDS17.html

## 案例展示：通过GAN来生成手写数字

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
from torch.utils.data import DataLoader

# 超参数
DOWNLOAD = True
use_GPU = torch.cuda.is_available()
Learning_Rate = 0.0003
EPOCH = 200

# 图像预处理
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

# 使用 MNIST 数据集
mnist = datasets.MNIST(root='./data/', train=True, transform=transform, download=DOWNLOAD)

# 包装成 DataLoader
data_loader = DataLoader(dataset=mnist, batch_size=100, shuffle=True)

# Discriminator Model
D = nn.Sequential(
    nn.Linear(784, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 1),
    nn.Sigmoid())

if use_GPU:
    D = D.cuda()

# Generator Model
G = nn.Sequential(
    nn.Linear(64, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 784),
    nn.Tanh())



if use_GPU:
    G = G.cuda()


# Loss and Optimizer
loss_fn = nn.BCELoss()

d_optimizer = torch.optim.Adam(D.parameters(), lr=Learning_Rate)
g_optimizer = torch.optim.Adam(G.parameters(), lr=Learning_Rate)


# Tensor to Variable
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

# start training
for epoch in range(EPOCH):
    for i, (images, _) in enumerate(data_loader):
        batch_size = images.size(0)
        images = to_var(images.view(batch_size, -1))

        # 给出正标签和负标签
        real_labels = to_var(torch.ones(batch_size))
        fake_labels = to_var(torch.zeros(batch_size))

        ###             训练 Discriminatro               ###
        # BCEloss(x, y) = -y * log(D(x)) - (1-y) * log(1 - D(x))
        # 当前 y = 1， 那么我们求得的Loss = -y * log(D(x)) 其中 x 来自于真实数据集
        outputs = D(images)
        d_loss_real = loss_fn(outputs, real_labels)
        real_score = outputs

        # 当前 y = 0， 那么我们求得的Loss = -(1-y) * log(1-D(x)) 其中 x 来自生成数据集
        z = to_var(torch.randn(batch_size, 64))
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = loss_fn(outputs, fake_labels)
        fake_score = outputs

        # 损失计算完成，开始方向传播
        d_loss = d_loss_real + d_loss_fake
        D.zero_grad()
        d_loss.backward()
        d_optimizer.step()


        ###             训练 Generator               ###
        z = to_var(torch.randn(batch_size, 64))
        fake_images = G(z)
        outputs = D(fake_images)


        # 本来我们应该是训练数据 使得 log(D(G(z))) 最大化，但是我们可以采取 minimzing log(1 - D(G(z)))的方式来进行代替
        # 这时候，我们将此时的标签看成正例即可。
        g_loss = loss_fn(outputs, real_labels)


        # 反向传播
        D.zero_grad()
        G.zero_grad()
        g_loss.backward()
        g_optimizer.step()


        if (i + 1) % 300 == 0:
            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, '
                  'g_loss: %.4f, D(x): %.2f, D(G(z)): %.2f'
                  % (epoch, 200, i + 1, 600, d_loss.data[0], g_loss.data[0],
                     real_score.data.mean(), fake_score.data.mean()))

    # Save real images
    if (epoch + 1) == 1:
        images = images.view(images.size(0), 1, 28, 28)
        save_image((images.data), './data/test_GAN/real_images.png')

    # Save sampled images
    fake_images = fake_images.view(fake_images.size(0), 1, 28, 28)
    save_image((fake_images.data), './data/test_GAN/fake_images-%d.png' % (epoch + 1))


# 保存训练参数
torch.save(G.state_dict(), './generator.pkl')
torch.save(D.state_dict(), './discriminator.pkl')


训练10次的效果图：

![GAN equal](./images/GAN/GAN_10.png)

训练50次的效果图：

![GAN equal](./images/GAN/GAN_50.png)

训练100次的效果图：

![GAN equal](./images/GAN/GAN_100.png)

训练150次的效果图：

![GAN equal](./images/GAN/GAN_150.png)

训练200次的效果图：

![GAN equal](./images/GAN/GAN_200.png)